<div style="text-align: center;">
<img src="images/stust.png" alt="STUST" class="center" style="width: 900px;"/>
</div>

<hr style="border:4px solid gray"> </hr>

<div style="text-align: center;">    
<br>    
    
# Control Arduino's LED & MQTT with Threads
# MQTT + I2C

</div>

<br>
<hr style="border:4px solid gray"> </hr>


<hr style="border:2px solid orange"> </hr>

# [What is the MQTT protocol](https://www.emqx.io/blog/what-is-the-mqtt-protocol)

# [How to use MQTT in Python (Paho)](https://www.emqx.io/blog/how-to-use-mqtt-in-python)
> Source: https://www.emqx.io/blog/how-to-use-mqtt-in-python

# [Use MQTT with Raspberry Pi](http://emqx.io/blog/use-mqtt-with-raspberry-pi)

### Install MQTT client library `paho-mqtt`
> Source: https://www.emqx.io/blog/use-mqtt-with-raspberry-pi

* Use the source code to install
>```bash
git clone https://github.com/eclipse/paho.mqtt.python 
cd paho.mqtt.python 
python3 setup.py install
>```

* Use pip3 to install
>```
pip3 install paho-mqtt
>```

### Free public MQTT broker provided by EMQ X:

>```text
Broker         : broker.emqx.io
TCP Port       : 1883
Websocket Port : 8083
>```

# [An Elegant Cross-platform MQTT 5.0 Client - MQTT X](https://mqttx.app/)
> Source: https://mqttx.app/


<hr style="border:2px solid orange"> </hr>

# Control Arduino-UNO's LEDs via I2C
# Arduino Sketch

<hr style="border:0.5px solid gray">

```C++
#include <Wire.h>

#define I2C_ADDR 11
#define R 11
#define Y 12
#define G 13

unsigned len;
byte msg[20];
byte cmd = 0;

// --------------------------------
void setup() {
  Wire.begin(I2C_ADDR);
  Serial.begin(9600);
  pinMode(R,OUTPUT); 
  pinMode(Y,OUTPUT); 
  pinMode(G,OUTPUT); 
  Serial.print("MQTT + I2C Homework 01: ");
  Serial.println(I2C_ADDR);
  Serial.println("=============================");
  delay(1000);    
  // register onReceive event services           
  Wire.onReceive(irsReceive);
}

// --------------------------------
void disp_msg() {
    Serial.print("Received data  : ");
    for(int i=0; i<len; i++) {
      Serial.print(msg[i]);
      Serial.print(" ");
    }
    Serial.println();  
    cmd = 0;
}

// --------------------------------
void LED(int pin, int stu) {
  digitalWrite(pin,stu);  
  cmd = 255;
}

// --------------------------------
void loop() {
  switch(cmd) {
    case 0   : break;
    case 1   : LED(R,0); break;
    case 2   : LED(R,1); break;
    case 3   : LED(Y,0); break;
    case 4   : LED(Y,1); break;
    case 5   : LED(G,0); break;
    case 6   : LED(G,1); break;
    case 255 :
    default  : disp_msg();
  }
  delay(0);
}

// --------------------------------
void irsReceive(int nb) {
  len = 0;
  while (Wire.available())
    msg[len++] = Wire.read();
  cmd = msg[0];  
}
```

<hr style="border:2px solid orange"> </hr>

# Control Arduino-UNO's LEDs via I2C

In [ ]:
import smbus2 as smbus 
from time import sleep
from random import randrange, random

I2C_ADDR = 11
PORT = 1
I2C = smbus.SMBus(PORT)

# LED-Control command
# 1, 3, 5 : turn off LED 'R', 'Y', 'G'
# 2, 4, 6 : turn on LED 'R', 'Y', 'G'
LED = {'R':[1,2], 'Y':[3,4], 'G':[5,6]}

# send LED-Control command to Arduino with parameters
# cmd : commands
# dat : additional parameters
def toArduino(cmd, dat=[]):
    global I2C
    bytes_msg = bytes(dat)
    print(f"send({cmd}, {dat})")
    I2C.write_i2c_block_data(I2C_ADDR, cmd, bytes_msg)
    sleep(0.1)

In [ ]:
# send turn-off-LED command to Arduino
def offAll():
    for led in LED.values():
        toArduino(led[0])

# send turn-on-LED command to Arduino
def onAll():
    for led in LED.values():
        toArduino(led[1])

In [ ]:
# ryg : 'R', 'Y', 'G'
# t   : blink t times
# t0  : off period
# t1  : on period
def blink(ryg, t=3, t0=0.2, t1=0.2):
    global R, Y, G
    while t>0:
        t=t-1
        toArduino(LED[ryg][1])
        sleep(t1)
        toArduino(LED[ryg][0])
        sleep(t0)

In [ ]:
# RED-to-GREEN sequence
# tr : RED period
# ty : YELLOW period
def r2g(tr=2, ty=0.4):
    toArduino(LED['R'][1])
    sleep(tr)
    toArduino(LED['Y'][1])
    sleep(ty)
    toArduino(LED['R'][0])
    toArduino(LED['Y'][0])
    toArduino(LED['G'][1])

# GREEN-to-RED sequence
# tg : GREEN period
def g2r(tg=3):
    toArduino(LED['G'][1])
    sleep(tg)
    toArduino(LED['R'][1])
    toArduino(LED['G'][0])

In [ ]:
# test cases
if __name__=="__main__":
    onAll()
    sleep(2)
    offAll()
    blink('R',5)
    blink('Y',5)
    blink('G',5,0.5,0.2)
    for i in range(3):
        r2g(3,0.4)
        g2r(4)
    offAll()

<hr style="border:2px solid orange"> </hr>

# MQTT Client with Threading


In [ ]:
import paho.mqtt.client as mqtt
import threading, os
from time import sleep

TOPIC  = f"P203-100"
BROKER = "broker.emqx.io"
PORT   = 1883
Client = None

# MQTT-Connected callback function 
def on_connect(client, userdata, flags, rc):
    global TOPIC
    print("Connected with result code "+str(rc))

# MQTT message retrieval callback function for the subscribed topics
def on_message(client, userdata, msg):
    global TOPIC
    topic = msg.topic
    payload = msg.payload.decode('utf-8')
    print(f"TOPIC({topic}) : MSG({payload})")
    if topic == TOPIC:
        if payload=='EXIT':
            client.disconnect()

def connect_mqtt():
    global Client
    # Create MQTT client and connect to the broker    
    Client = mqtt.Client()
    Client.on_connect = on_connect
    Client.on_message = on_message
    # client.username_pw_set("user-name","password")
    Client.connect(BROKER, PORT, keepalive=60)
    # Clear the retained message send by Kittenblock
    Client.publish(topic=TOPIC, payload=None, retain=True)
    # Subscribe interested topic(s)
    Ｃlient.subscribe(TOPIC)

# Main Loop
def main():
    global LOOP, Client
    Client.loop_forever()
    print("main: Bye!")
    LOOP = False

# Dummy task for thread demonstration
def task01():
    global LOOP
    while LOOP:
        sleep(1)
    print("task01: Bye!")
    
if __name__ == "__main__":
    connect_mqtt()
    LOOP = True
    t = threading.Thread(target = task01)
    t.start()
    main()
    t.join()
    print("All done.") 

<hr style="border:0.5px solid gray"> 

<br><hr style="border:3px solid red"> </hr>
<div style="text-align: left;">         
    
# *Homework Assignment*

</div>
<hr style="border:3px solid red"> </hr>

## Add problem here

<hr style="border:2px solid orange"> </hr>
<br>

<div style="text-align: left;">
<img src="images/break-yang-tr.png" alt="Break" class="center" style="width: 500px;"/>
</div>
